# Faces recognition using eigenfaces and SVMs


In [1]:
from sklearn.model_selection import GridSearchCV, StratifiedKFold
from sklearn.decomposition import PCA
from sklearn.svm import SVC

import numpy as np

In [2]:
#load data from train and test set
X_train = np.load('X_train.npy')
y_train = np.load('y_train.npy')
#X_test = np.load('X_test.npy')
X_test = np.load('corrected.lfw.heldout.npy')


In [3]:
#number of eigenfaces to extract
n_components = 125

#height and width of X_train
h = 50
w = 37

pca = PCA(n_components=n_components, svd_solver='randomized',
          whiten=True).fit(X_train)
eigenfaces = pca.components_.reshape((n_components, h, w))

#X_train and X_test after transform to eigenfaces features
X_train_pca = pca.transform(X_train)
X_test_pca = pca.transform(X_test)


In [4]:
#parameters to search for the best estimator
param_grid = [
{'C': [0.1,1,10,100,1e3,5e3], 'kernel':['linear']},
{'C': [0.1,1,10,100,1e3, 5e3], 'gamma': ['auto', 0.001, 0.005, 0.01, 0.05,0.1,0.5], 'kernel': ['rbf', 'sigmoid']},
{'C': [0.1,1,10,100,1e3, 5e3],'gamma': ['auto', 0.001, 0.005, 0.01, 0.05,0.1,0.5],
 'kernel': ['poly'], 'degree': [2,3]}]


In [5]:
print("Searching for best estimator from given parameters")
cv = StratifiedKFold(n_splits = 10)

clf = GridSearchCV(SVC(class_weight='balanced'), param_grid, cv=cv, n_jobs=-1)
clf = clf.fit(X_train_pca, y_train)

fo = open('estimator.txt', 'w')
fo.write(str(clf.best_estimator_) + "\n")
fo.close()
print(clf.best_estimator_)
print(clf.score(X_train_pca, y_train))

y_pred = clf.predict(X_test_pca)
fo = open('heldout.txt', 'w')
fo.write("ImageId,PredictedClass"+"\n")
for i in range(y_pred.size):
    fo.write(str(i) + "," + str(y_pred[i]) + "\n")
fo.close()

Searching for best estimator from given parameters
SVC(C=1, cache_size=200, class_weight='balanced', coef0=0.0,
  decision_function_shape=None, degree=3, gamma=0.005, kernel='rbf',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False)
0.989648033126
